In [1]:
#####################################
#Leaf classifier + Lesion classifier#
#####################################

#########
#outputs#
#########

# - res.xlsx reports the measures for each individual leaf

# - *img_name*-NGRDI/TGI/lesion_sizes.txt reports each grouping's mean for each measure and 
#    one way ANOVA results

# - *img_name*_ids.jpg             shows each leaf's ID

# - *img_name*_kmeans_grouping.jpg shows kmeans grouping

# - *img_name*_mask.png shows the leaf classifier's prediction, and in each Class_X
#   there is a Masks folder containing the lesion classifier's predictions

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#############################################################################################
#run this cell by removing quotes to install dependencies which are not included in Anaconda#
#############################################################################################
'''
import sys
!pip install --prefix {sys.prefix} fastai==1.0.59
!pip install --prefix {sys.prefix} torch==1.4.0
!pip install --prefix {sys.prefix} autoimpute
!pip install --prefix {sys.prefix} opencv-python
'''

In [12]:
########
#import#
########
from inference_funcs_GLI_Feb10 import Run_Lesion_Inference, Run_Leaf_Inference, compile_time_series_results
import os
import glob
from fastai import *
import torch
import cv2
from UNet_init_funcs import *
import warnings
home_dir = os.getcwd()

#for silencing imageio warnings when saving lesion mask images.
import imageio.core.util
def silence_imageio_warning(*args, **kwargs):
    pass
imageio.core.util._precision_warn = silence_imageio_warning

In [8]:
################
#set parameters#
################
defaults.device = torch.device('cpu') #cpu or gpu utilization
img_dir = home_dir + '/Images' #folder where images are located

leaf_fnames = glob.glob(img_dir+'/*.png')   #choose files for inference - expects list of file names
                                            #make sure images are in order for time series analysis

clust_by = 'x'                        #k means clustering by row ('y') or column ('x')

n_classes = [6] * len(leaf_fnames) #number of leaf groups - expects list of numbers same length as leaf_fnames
                                      #for time series data can set it as e.g. [5] * len(leaf_fnames)

time_series_analysis = 'n'            #'y' or 'n'
    
overlap_ids = [] * len(leaf_fnames)   #to flag up potential overlaps, which allows for downstream correction
                                      #if none provided, then thresholding is used

out_dir = 'out'        #folder to save results in 
bright_norm = None

############################## 
#time series analysis settings#
##############################
use_t0_as_reference = 'n'             #'y' or 'n'
                                      #if 'y', will propagate time point 0's image mask throughout the time series

ignore_fnames = []      #will ignore indicated images, and interpolate the missing time point
missing_tps = []        #will interpolate the missing time points indicated
impute_method = 'interpolate' #method of interpolation, use None if imputation not needed
                              #other imputation methods can be found in autoimpute's docs
ts_dir = 'time_series_res' #folder for time series results

start_time = 0 
end_time = 73
interval = 1

if time_series_analysis == 'y' and len(ignore_fnames) != 0:
    for fn in ignore_fnames:
        ind = leaf_fnames.index(fn)
        missing_tps.append(ind)
    for fn in ignore_fnames:
        leaf_fnames.remove(fn)
        
t0_fpath = out_dir+'/'+os.path.split(leaf_fnames[0])[1][:-4] + '_res'
t = 0

In [26]:
###################
#Leaf segmentation#
###################
for (fpath,n) in zip(leaf_fnames,n_classes):
    Run_Leaf_Inference(fpath, clust_by,n, time_series_analysis, use_t0_as_reference, overlap_ids, out_dir, bright_norm,t, t0_fpath)
    t += 1

KeyboardInterrupt: 

In [17]:
#At this stage can correct for any masks, and running the above cell again 
#will trigger the corrected_mask_processing function.

In [15]:
#######################################################################
#run this cell to create the directory paths for the lesion classifier#
#######################################################################
#no need to change anything if directly running after leaf inference

leaf_res_paths = []
for fn in leaf_fnames:
    fname = os.path.split(fn)[1]
    leaf_dir = fname[:-4] + '_res'
    leaf_res_paths.append(os.path.join(out_dir, leaf_dir))

#creates the time series results dataframe to store results from lesion prediction
ts_res = []
if time_series_analysis == 'y':
    #for each group make a df 
    for g in range(n_classes[0]):
        ts_df = pd.DataFrame(index = ['lesion_size(ratio)','GLI','NGRDI','lesion_size(pn)'], columns = range(start_time, end_time,interval))
        ts_res.append(ts_df)

leaf_res_paths = ['out/OW_valid_res']
n_classes = [1]

In [16]:
#####################
#Lesion segmentation#
#####################
t=0
for (fn,n) in zip(leaf_res_paths,n_classes):
    ts_res = Run_Lesion_Inference(fn,n,time_series_analysis,bright_norm,ts_res,t)
    print(t)
    t += 1

if time_series_analysis == 'y':
    compile_time_series_results(time_series_analysis, ts_res, out_dir, ts_dir, missing_tps, impute_method, lesion_learner_path = 'Lesion_UNet/final_64')

0
